In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.2860,), (0.3530,))
])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                             download=True, transform=transform)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                            download=True, transform=transform)

train_data, val_data = train_test_split(trainset, test_size=0.1, random_state=42)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,
                                           shuffle=True, )
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64,
                                         shuffle=False, )
test_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                          shuffle=False, )

model = resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(model.fc.in_features, 10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    val_loss = running_loss / len(val_loader)
    val_acc = 100. * correct / total
    precision = precision_score(all_targets, all_preds, average='macro')
    recall = recall_score(all_targets, all_preds, average='macro')
    f1 = f1_score(all_targets, all_preds, average='macro')
    conf_matrix = confusion_matrix(all_targets, all_preds)
    return val_loss, val_acc, precision, recall, f1, conf_matrix


epochs = 5
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, precision, recall, f1, conf_matrix = validate(model, val_loader, criterion, device)
    print(f'Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, '
          f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')
    print("Confusion Matrix:")
    print(conf_matrix)

test_loss, test_acc, precision, recall, f1, conf_matrix = validate(model, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%, '
      f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')
print("Confusion Matrix:")
print(conf_matrix)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Train Loss: 0.4990, Train Acc: 81.89%, Val Loss: 0.4787, Val Acc: 82.35%, Precision: 0.8485, Recall: 0.8254, F1-score: 0.8252
Confusion Matrix:
[[459   0  35   2   1   0  99   0   1   0]
 [  4 590   3  10   0   0   1   0   0   0]
 [  8   0 560   2  13   0  28   0   0   0]
 [ 60   3  13 462   7   0  40   0   0   2]
 [  1   0 126  29 375   0  93   0   3   0]
 [  1   0   0   0   0 569   0  40   5   6]
 [ 67   1 102   7   8   1 429   0   4   0]
 [  0   0   0   0   0   4   0 543   2   1]
 [  5   1   5   1   1   0  17   0 560   0]
 [  0   0   0   0   0   1   0 194   1 394]]
Epoch 2: Train Loss: 0.3494, Train Acc: 87.60%, Val Loss: 1.4503, Val Acc: 70.62%, Precision: 0.7409, Recall: 0.6998, F1-score: 0.6728
Confusion Matrix:
[[507  16   3  19   8   2  42   0   0   0]
 [  0 589   0  15   3   0   1   0   0   0]
 [  3  53 246   6 185   0 118   0   0   0]
 [ 17  26   0 453  75   1  12   0   3   0]
 [  0  22   2   3 576   1  21   0   2   0]
 [  0  41   1   0   0 577   0   0   0   2]
 [112